### 1. 데이터 준비

In [ ]:
import torch  ##PyTorch 라이브러리 임포트 ( 딥러닝 프레임워크 )
from torch.utils.data import Dataset, DataLoader  ## 데이터셋과 데이터로더를 위한 모듈 임포트
import pandas as pd  ## 데이터프레임 처리를 위한 pandas 라이브러리 임포트
import numpy as np  ## 배열 연산을 위한 numpy 라이브러리 임포트

## ASL 데이터셋 클래스 정의 (PyTorch의 Dataset 클래스를 상속)
class ASL_Dataset(Dataset):  
  def __init__(self, path):
    super(ASL_Dataset, self).__init__()  ## 부모 클래스 ( Dataset )의 초기화 메서드 호출

## CSV 파일을 읽어 데이터프레임으로 로드
    df = pd.read_csv(path)
    self.y = df['label']  ## 'label' 열을 추출 -> 타겟 변수 ( y )로 설정
    del df['label']  ## 데이터프레임에서 'label' 열 삭제 ( 입력 데이터만 남김 )
    self.x = df.values  ## 나머지 데이터를 numpy 배열로 변환하여 입력 변수 ( x )로 설정


## 데이터에서 특정 인덱스의 샘플을 반환하는 메서드
  def __getitem__(self, index):
    x_sample = np.uint8( self.x[index].reshape(28,28) )  ## 입력 데이터를 28X28 크기의 이미지로 재구성 ( uint8 타입으로 변경 )
    x_tensor = torch.tensor ( x_sample[np.newaxis,:,:] ).float()  ## 단일 채널 ( 1차원 )을 추가하여 텐서로 변환 ( 형식 : [1, 28, 28]), float 타입으로 변환
    y_tensor = torch.tensor ( self.y[index] )  ## 해당 인덱스의 레이블을 텐서로 변환
    return x_tensor, y_tensor  ## 입력 텐서와 레이블 텐서를 튜플로 반환


## 데이터셋의 전체 길이 ( 샘플 수 )를 반환하는 메서드
  def __len__(self):
    return self.x.shape[0]  ## 입력 데이터의 행 개수( 샘플 수 ) 반환환

In [ ]:
## 훈련 데이터셋 객체 생성 ( sign_mnist_train.csv 파일을 기반으로 ASL_Dataset 인스턴스 생성 )
train_data = ASL_Dataset("sign_mnist_train.csv")

## 검증 데이터셋 객제 생성 ( sign_mnist_valid.csv 파일을 기반으로 ASL_Dataset 인스턴스 생성 )
valid_data = ASL_Dataset("sign_mnist_valid.csv")

## 훈련 데이터로드 생성
## - dataset : 생성할 데이터셋 ( train_data )
## - batch_size :  한 번에 처리할 샘플 수 ( 1000개 )
## - shuffle : 데이터 무작위로 섞을지 여부 ( True로 설정하여 섞음 )
train_loader = DataLoader(dataset=train_data, batch_size=1000, shuffle=True)

## 검증 데이터로더 생성
## - datastet : 사용할 데이터셋 ( valid_data )
## - batch_size : 한 번에 처리할 샘플 수 ( 1000개 )
## - shuffle : 데이터 무작위로 섞을지 여부 ( False로 설정하여 섞지 않음 )
valid_loader = DataLoader(dataset=valid_data, batch_size=1000, shuffle=False)

### 2. 뉴럴네트워크 모델링

In [19]:
import torch.nn as nn
class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
      self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
      self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
      self.relu = nn.ReLU()
      self.pool = nn.MaxPool2d(2,2)
      self.fc = nn.Linear(7*7*32, 24)

  def forward(self, x):
    x = self.relu( self.conv1(x) )
    x = self.pool(x)   # 14X14X16
    x = self.relu( self.conv2(x) )
    x = self.pool(x)   # 7X7X32
    x = self.relu( self.conv3(x) )   # 7x7x32

    x = x.view(-1, 7*7*32)
    x = self.fc(x)
    return x

In [20]:
convnet = Net()

### 3.손실함수와 최적화함수의 지정

In [23]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

### 4. 뉴럴네트워크 학습

In [24]:
for epoch in range(5):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()

        inputs, labels = data
        pred = convnet(inputs)

        loss = loss_function(pred, labels)

        loss.backward()

        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, " , Loss:", loss_val)

Epoch: 1  , Loss: 139.78275990486145
Epoch: 2  , Loss: 41.181666135787964
Epoch: 3  , Loss: 12.290689006447792
Epoch: 4  , Loss: 4.458873055875301
Epoch: 5  , Loss: 1.911327749490738


### 5. 뉴럴네트워크 평가

In [26]:
pred_list = []
label_list = []
for itr, data in enumerate(valid_loader):
  inputs, labels = data
  pred_test = convnet(inputs)
  pred_category = torch.argmax(pred_test, dim=1)

  pred_list = pred_list + list(pred_category)
  label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list) ==np.array(label_list) )
print("Test accuracy: ", accu)

Test accuracy:  0.8623814835471277
